 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/importing%20notebooks.ipynb#Importing-Jupyter-Notebooks-as-Modules" data-toc-modified-id="Importing-Jupyter-Notebooks-as-Modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Jupyter Notebooks as Modules</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/importing%20notebooks.ipynb#Notebook-Loader" data-toc-modified-id="Notebook-Loader-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Notebook Loader</a></span></li><li><span><a href="http://localhost:8888/notebooks/importing%20notebooks.ipynb#The-Module-Finder" data-toc-modified-id="The-Module-Finder-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The Module Finder</a></span></li><li><span><a href="http://localhost:8888/notebooks/importing%20notebooks.ipynb#Register-the-hook" data-toc-modified-id="Register-the-hook-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Register the hook</a></span></li><li><span><a href="http://localhost:8888/notebooks/importing%20notebooks.ipynb#Aside:-displaying-notebooks" data-toc-modified-id="Aside:-displaying-notebooks-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Aside: displaying notebooks</a></span></li></ul></li></ul></div>

# Importing Jupyter Notebooks as Modules

http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Importing%20Notebooks.ipynb

In [1]:
import io, os, sys, types

In [2]:
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

Import hooks typically take the form of two objects:

1. a Module **Loader**, which takes a module name (e.g. 'IPython.display'), and returns a Module
2. a Module **Finder**, which figures out whether a module might exist, and tells Python what **Loader** to use



In [3]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
            


## Notebook Loader

Here we have our Notebook Loader. It's actually quite simple - once we figure out the filename of the module, all it does is:

   1. load the notebook document into memory
   2. create an empty Module
   3. execute every cell in the Module namespace

Since IPython cells can have extended syntax, the IPython transform is applied to turn each of these cells into their pure-Python counterparts before executing them. If all of your notebook cells are pure-Python, this step is unnecessary.


In [4]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
            
              for cell in nb.cells:
                    
                    if cell.cell_type == 'code':
                        # transform the input to executable Python
                        code = self.shell.input_transformer_manager.transform_cell(cell.source)
                        # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


## The Module Finder

The finder is a simple object that tells you whether a name can be imported, and returns the appropriate loader. All this one does is check, when you do:

```python 
import mynotebook
```

it checks whether ```mynotebook.ipynb``` exists. If a notebook is found, then it returns a NotebookLoader.

Any extra logic is just for resolving paths within packages.


In [5]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


## Register the hook

Now we register the NotebookFinder with ```sys.meta_path```

In [6]:
sys.meta_path.append(NotebookFinder())

After this point, my notebooks should be importable.

Let's look at what we have in the CWD:


In [7]:
ls data/

__init__.py  pascal_voc_parser.ipynb  pascal_voc_parser-pyfile.py  __pycache__/


In [8]:
from data import pascal_voc_parser as p

importing Jupyter notebook from /home/abanihi/Documents/Github/keras-faster-rcnn/data/pascal_voc_parser.ipynb
Parsing annotation files....
Parsing annotation files Finished without error!



## Aside: displaying notebooks

Here is some simple code to display the contents of a notebook with syntax highlighting, etc.


In [10]:


from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.display import display, HTML

formatter = HtmlFormatter()
lexer = PythonLexer()

# publish the CSS for pygments highlighting
display(HTML("""
<style type='text/css'>
%s
</style>
""" % formatter.get_style_defs()
))



In [11]:
def show_notebook(fname):
    """display a short summary of the cells of a notebook"""
    with io.open(fname, 'r', encoding='utf-8') as f:
        nb = read(f, 4)
    html = []
    for cell in nb.cells:
        html.append("<h4>%s cell</h4>" % cell.cell_type)
        if cell.cell_type == 'code':
            html.append(highlight(cell.source, lexer, formatter))
        else:
            html.append("<pre>%s</pre>" % cell.source)
    display(HTML('\n'.join(html)))

In [12]:
show_notebook("data/pascal_voc_parser.ipynb")